In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+rocm5.6
True


In [2]:
! git clone https://github.com/aurorabuilder/elements

fatal: destination path 'elements' already exists and is not an empty directory.


In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.llms import LlamaCpp
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_openai import ChatOpenAI

In [3]:
# model = LlamaCpp(
#     model_path="models/neural-chat-7b-v3-3.Q5_K_M.gguf",
#     temperature=0.1,
#     max_tokens=512,
#     top_p=0.95,
#     # callback_manager=callback_manager,
#     verbose=True, # Verbose is required to pass to the callback manager
#     n_gpu_layers=-1,
#     echo=True,
#     n_ctx=1024*32,
#     stop=["### System", "### User", "### Assistant", "User:", "Player:", "DM:"],
# )

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 ROCm devices:
  Device 0: AMD Radeon RX 6800, compute capability 10.3, VMM: no
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/neural-chat-7b-v3-3.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = intel_neural-chat-7b-v3-3
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.f

In [21]:
import openai

# openai.api_key = "EMPTY"
# openai.base_url = "http://localhost:5000/v1/"

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=512, 
                   openai_api_base="http://localhost:5000/v1/", # Remove this to use the actual OpenAI API
                   streaming=True,)

In [4]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [5]:
from langchain.document_loaders import PyPDFLoader, UnstructuredXMLLoader
from langchain_community.document_loaders.unstructured import UnstructuredBaseLoader


import xml.etree.ElementTree as ET
from typing import List, Any

class XMLRPGClassLoader(UnstructuredBaseLoader):
    """Custom Loader for RPG class XML files, focusing on detailed features/abilities."""

    def __init__(
        self,
        file_path: str,
        mode: str = "elements",
        **unstructured_kwargs: Any,
    ):
        """Initialize with XML file path."""
        self.file_path = file_path
        super().__init__(mode=mode, **unstructured_kwargs)

    def _parse_xml(self) -> ET.ElementTree:
        """Parse the XML file and return the root element."""
        tree = ET.parse(self.file_path)
        return tree.getroot()

    def _get_elements(self) -> List[dict]:
            """Extract elements from the XML file."""
            root = self._parse_xml()
            elements = []

            for element in root.findall('.//element'):
                element_data = {}
                element_texts = []

                # Element name and type
                name = element.get('name', 'Unnamed Feature')
                element_type = element.get('type', 'Unknown Type')
                element_texts.append(name)
                element_data['type'] = element_type

                # Extracting description, sheet, and supports text
                for tag in ['description', 'sheet', 'supports']:
                    text_content = element.find(tag)
                    if text_content is not None:
                        text = ''.join(text_content.itertext()).strip()
                        element_texts.append(text)

                # Combine all texts and add metadata
                element_data['text'] = "\n\n".join(element_texts)
                elements.append(element_data)

            return elements

    def _get_metadata(self) -> dict:
        """Get metadata for the XML file."""
        return {"source": self.file_path, "file_type": "xml"}

# Example usage
# loader = XMLRPGClassLoader(file_path="elements/core/players-handbook/classes/class-barbarian.xml")
# docs = loader.load()
# docs[0]

In [6]:
import os 
import glob  
# Define the directory path 
directory_path = 'elements/core/players-handbook/classes'  
# Initialize a list to store the XML file contents as strings 
xml_strings = []  
# Use the glob module to recursively find XML files in the directory and its subfolders 
xml_files = glob.glob('*.xml', root_dir=directory_path, recursive=True)

documents = []

for xml_file in xml_files:
    loader = XMLRPGClassLoader(os.path.join(directory_path, xml_file))
    documents.extend(loader.load())
# Loop through each XML file and read its content as a string 
# for xml_file in xml_files:     
#     with open(os.path.join(directory_path, xml_file), 'r', encoding='utf-8') as file:         
#         xml_string = file.read()         
#         xml_strings.append(xml_string)  
# Now, the xml_strings list contains the contents of all XML files as strings

In [7]:
documents[0]

Document(page_content="{'type': 'Class', 'text': 'Cleric\\n\\nArms and eyes upraised toward the sun and a prayer on his lips, an elf begins to glow with an inner light that spills out to heal his battle-worn companions.\\n\\t\\t\\tChanting a song of glory, a dwarf swings his axe in wide swaths to cut through the ranks of orcs arrayed against him, shouting praise to the gods with every foe’s fall.\\n\\t\\t\\tCalling down a curse upon the forces of undeath, a human lifts her holy symbol as light pours from it to drive back the zombies crowding in on her companions.\\n\\t\\t\\tClerics are intermediaries between the mortal world and the distant planes of the gods. As varied as the gods they serve, clerics strive to embody the handiwork of their deities. No ordinary priest, a cleric is imbued with divine magic.\\n\\t\\t\\tHEALERS AND WARRIORS\\n\\t\\t\\tDivine magic, as the name suggests, is the power of the gods, flowing from them into the world. Clerics are conduits for that power, manife

### Sentence splitting

from langchain.text_splitter import CharacterTextSplitter, SentenceTransformersTokenTextSplitter

# text_splitter = CharacterTextSplitter(chunk_size=128, chunk_overlap=30, separator="\n")
text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0)
docs = text_splitter.split_documents(documents=documents)
docs[1]

### Vector store
Let's load the document inside the FAISS vector store. Pay attention that the metadata will be discarded, so include everything that would be relevant to the model inside the text.

In [8]:
document_texts = [doc.page_content for doc in documents]
vectorstore = FAISS.from_texts(texts=document_texts, embedding=hf)
retriever = vectorstore.as_retriever(search_kwargs={"k": 100})

### Reranking
Vector stores' performance can be increased by reranking the documents extracted with the embeddings by using a dedicated reranking model. The output of such a model will be an ordered subset of the input documents retrieved by the vector store

In [9]:
from __future__ import annotations
from typing import Dict, Optional, Sequence
from langchain.schema import Document
from langchain.pydantic_v1 import Extra, root_validator

from langchain.callbacks.manager import Callbacks
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor

from sentence_transformers import CrossEncoder
# from config import bge_reranker_large

class BgeRerank(BaseDocumentCompressor):
    model_name: str = 'BAAI/bge-reranker-large'  
    """Model name to use for reranking."""    
    top_n: int = 10   
    """Number of documents to return."""
    model: CrossEncoder = CrossEncoder(model_name, device="cuda")
    """CrossEncoder instance to use for reranking."""

    def bge_rerank(self,query,docs):
        model_inputs =  [[query, doc] for doc in docs]
        scores = self.model.predict(model_inputs)
        results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return results[:self.top_n]


    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using BAAI/bge-reranker models.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        doc_list = list(documents)
        _docs = [d.page_content for d in doc_list]
        results = self.bge_rerank(query, _docs)
        final_results = []
        for r in results:
            doc = doc_list[r[0]]
            doc.metadata["relevance_score"] = r[1]
            final_results.append(doc)
        return final_results

In [10]:
from langchain.retrievers import ContextualCompressionRetriever

compressor = BgeRerank(top_n=5)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [11]:
compression_retriever.get_relevant_documents(query="Barbarian history proficiency")
context = str(compression_retriever.get_relevant_documents(query="Barbarian history proficiency"))

# Sanitize context to be used in a formattable string
context = context.replace("{", "{{").replace("}", "}}")

In [22]:
template = """### System:
Answer the user's question based only on the following context:
{context}

### User: Question: {question}

### Assistant:
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
chain.invoke("What's a bard's most important skill?")

"\nA bard's most important skill lies in their ability to weave magic through words and music. Their versatility, knowledge, and talent in various subjects make them masters of performance, speech, and the magic they contain. Bards are known for their skill in inspiring allies, demoralizing foes, manipulating minds, creating illusions, and even healing wounds through their spells and powers. Their strength lies in their sheer versatility and the depth of knowledge they possess, making them valuable assets in any adventuring party."